In [1]:
import csv
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import re
import math

In [2]:
df= pd.read_csv('data/2019Transcriptions.csv')

In [3]:
#df.head()

In [4]:
copy=df
#copy.head()
# "copy" is now the working copy

In [5]:
copy.drop(['Transcriber'], axis=1, inplace=True)

In [6]:
#copy.head()

#  Parcels Table


In [7]:
parcels=df[['Block #', 'Parcel #','Ward #','Land use']]
# pulling in attributes for Parcels Entity 

In [8]:
parcels=parcels[parcels['Block #'].str.contains("  ")==False]

In [9]:
parcels['Block #'].dtypes

dtype('O')

In [10]:
parcels.insert(0,'Parcel_ID', range(1,1+len(parcels)))

In [11]:
parcels.head()

,Parcel_ID,Block #,Parcel #,Ward #,Land use
0,1,34,7,4,Commercial
1,2,59,14,4,Residential
2,3,59,15,4,Residential
3,4,59,16,4,Residential
4,5,59,18,4,Residential


In [12]:
parcels.columns = ["parcel_id", "block_num", "parcel_num", "ward_num", "land_use"]

In [13]:
parcels.to_csv('output/parcels.csv', index=False)

In [14]:
parcels.to_csv('db_output/parcels.csv', index=False)

In [15]:
parcels. shape

(937, 5)

In [16]:
parcel_id=parcels[['parcel_id']]

In [17]:
# pulling attributes for the Address entity
addresses= copy[['Street number', 'Street address']]

In [18]:
# connecting to "parcel_id" as an attribute in the Address entity since it will be the forigen key to Parcels Entity
addresses=addresses.join(parcels[['parcel_id']])

In [19]:
addresses.head()

,Street number,Street address,parcel_id
0,215,Southside Ave.,1.0
1,22,Hibernia Rd.,2.0
2,20,Hibernia Rd.,3.0
3,16,Hibernia Rd.,4.0
4,NaN,Hibernia Rd.,5.0


In [20]:
addresses.shape

(939, 3)

In [21]:
addresses.head()

,Street number,Street address,parcel_id
0,215,Southside Ave.,1.0
1,22,Hibernia Rd.,2.0
2,20,Hibernia Rd.,3.0
3,16,Hibernia Rd.,4.0
4,NaN,Hibernia Rd.,5.0


In [22]:
# function to take in an object (street num) and split the rows 
def splitRow(table):
#     columns are the values from the colunms of the object passed
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
#         if there is a break within the street num, split the strings then add another row withing that particular cell
        if '\n' in str(row['Street number']):
            data = row['Street number'].split('\n')
            for d in data:
                newrow = row
                newrow['Street number'] = d
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            size+=1
    return newframe

In [23]:
#spliting street num and addresses, now "test" is our modified and split address object
test= splitRow(addresses)
test
##s = test[test['Street address'] == 'Hillard Ave.']
##x = addresses[addresses['Street address'] == 'Hillard Ave.']
##print(x)


C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Street number,Street address,parcel_id
0,215,Southside Ave.,1.0
1,22,Hibernia Rd.,2.0
2,20,Hibernia Rd.,3.0
3,16,Hibernia Rd.,4.0
4,NaN,Hibernia Rd.,5.0
...,...,...,...
943,153,Blanton St.,935.0
944,NS of,Smith St.,936.0
945,171,Blanton St.,937.0
946,NaN,NaN,NaN


In [24]:
test.insert(0,'ID',range(1,1+len(test)))

In [25]:
test.head()

,ID,Street number,Street address,parcel_id
0,1,215,Southside Ave.,1.0
1,2,22,Hibernia Rd.,2.0
2,3,20,Hibernia Rd.,3.0
3,4,16,Hibernia Rd.,4.0
4,5,NaN,Hibernia Rd.,5.0


In [26]:
# setting the datatype for parcel_id as int
test.parcel_id = test["parcel_id"]

In [27]:
test.columns = ["id", "st_num", "st_name", "parcel_id"]

In [28]:
# saving the Entity address
test.to_csv('output/addresses.csv', index=False)

In [29]:
test.head()

,id,st_num,st_name,parcel_id
0,1,215,Southside Ave.,1.0
1,2,22,Hibernia Rd.,2.0
2,3,20,Hibernia Rd.,3.0
3,4,16,Hibernia Rd.,4.0
4,5,NaN,Hibernia Rd.,5.0


In [30]:
test.to_csv('db_output/addresses.csv', index=False)

# People Table

In [31]:
# retrieving attributes for the People Entity
people=copy[['Previous Transfer of Deed People Involved','Decision People Involved','Final Transfer of Deed People Involved','Appraiser Name','Offer Made People Involved']]

In [32]:
people.head()

,Previous Transfer of Deed People Involved,Decision People Involved,Final Transfer of Deed People Involved,Appraiser Name,Offer Made People Involved
0,NaN,NaN,NaN,NaN,NaN
1,"A. G. Carver, Jr.\nEvelyn Carter\n\nTroy Barne...",Troy Barnett\nHattie Barnett\n\nHACA,Troy Barnett\nHattie Barnett\n\nHACA,Francis Naeger\nAlan Butterworth\nCharles W. B...,Troy Barnett\nHattie Barnett\n\nHACA
2,William P. Shuferd\n\nLeslie Mae Wilson,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,Francis Naeger\nAlan Butterworth\nCharles W. B...,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...
3,Albert Kodack\nAnn Kodack\n\nH. K. Lee,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Charles W. Bradshaw \nFrancis Naeger\nAlan But...,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...
4,Bostic\n\nGwyn Edwards,"Irwin Monk, et als\n\nHACA",Irwin Monk\nW. Clarence Monk\nInez Monk\nFred ...,Roy Moore\nH.E. Johnson,Irwin Monk\nW. Clarence Monk\nInez Monk\nFred ...


In [33]:
# converting the rows into colunmns 
people = people.stack().reset_index()

In [34]:
people.head()

,level_0,level_1,0
0,1,Previous Transfer of Deed People Involved,"A. G. Carver, Jr.\nEvelyn Carter\n\nTroy Barne..."
1,1,Decision People Involved,Troy Barnett\nHattie Barnett\n\nHACA
2,1,Final Transfer of Deed People Involved,Troy Barnett\nHattie Barnett\n\nHACA
3,1,Appraiser Name,Francis Naeger\nAlan Butterworth\nCharles W. B...
4,1,Offer Made People Involved,Troy Barnett\nHattie Barnett\n\nHACA


In [35]:
people.columns = ["a","b","names"]

In [36]:
# dropping level_0 and level_1 (title) 
people = people.drop(['a', 'b'], axis=1)

In [37]:
# function to take in an object and split the rows 
def splitRows(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
        print(row)
        if '\n' in str(row['names']):
            names = row['names'].split('\n')
            print(names)
            for n in names:
                print (n)
                newrow = row
                newrow['names'] = n
                newframe.loc[size]=newrow
                size+=1
                print(newrow)
        else:
            newframe.loc[size]=row
            size+=1
        break
#     print("final:")
#     print (newframe)
    return newframe

In [38]:
def name_cleaner(name): 
    k = re.sub(r'[0-9][.]', '', name)
    k = re.sub(r'[\(\)\-\&]', '', k)
    k = re.sub(r'[0-9]', '', k)
    k = re.sub(r'[.,]', ' ', k)
    k = re.sub('  ', ' ', k)
    k = k.strip()
    return k

In [39]:
def date_cleaner(date):
    k = re.sub(r'[0-9][.]', '', date)
    k = k.strip()
    return k

In [40]:

def name_organizer(table):
    clean_names=[]
#     looping through the same row 
    for index, row in table.iterrows():
        names = str(row['names'])
        
        if '\n' in names:
            arr = names.split('\n')
            arr = [i for i in arr if i is not ""]
            for n in arr:
                k = name_cleaner(n)
                if k == '': 
                    continue
                clean_names.append(k)
        else:
            k = name_cleaner(names)
            if k == '': 
                continue
            clean_names.append(k)
    return clean_names

In [41]:
# passing people through the function name_organizer to get multiple rows of peoples names 
people=name_organizer(people)

In [42]:
# newframe= declaring a new table with a column "name" 
newframe=pd.DataFrame(columns=['name'])


In [43]:
# assigning each name to the table newframe 
newframe['name'] = pd.Series(people)
newframe.head()

,name
0,A G Carver Jr
1,Evelyn Carter
2,Troy Barnett
3,Hattie Barnett
4,Troy Barnett


In [44]:
newframe = newframe.sort_values(by='name', ascending=True)
newframe.head()

,name
8958,#NAME?
1126,*Not Legable
1209,A Blomberg
13180,A C Brandl
13181,A C Brandl Jr


In [45]:
newframe['id'] = range(len(newframe['name']))
newframe.head()

,name,id
8958,#NAME?,0
1126,*Not Legable,1
1209,A Blomberg,2
13180,A C Brandl,3
13181,A C Brandl Jr,4


In [46]:
newframe.drop(index=0)

,name,id
8958,#NAME?,0
1126,*Not Legable,1
1209,A Blomberg,2
13180,A C Brandl,3
13181,A C Brandl Jr,4
...,...,...
2997,lamar Gudger,14036
3010,lamar Gudger,14037
2503,tecora Cunnigham,14038
2488,tecora Cunnigham,14039


In [47]:
newframe.index = range(0,len(newframe))

In [48]:
newframe.columns = ["name", "person_id"]

In [49]:
newframe["person_id"] = newframe.index + 1

In [50]:
newframe.head()

,name,person_id
0,#NAME?,1
1,*Not Legable,2
2,A Blomberg,3
3,A C Brandl,4
4,A C Brandl Jr,5


In [51]:
newframe.to_csv('output/people_count.csv',  index=False)

## Event Association Table

In [52]:
event_assoc=copy[['Decision People Involved','Decision Role']]

In [53]:
event_assoc.head()

,Decision People Involved,Decision Role
0,NaN,NaN
1,Troy Barnett\nHattie Barnett\n\nHACA,Owner\n\nPurchaser
2,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,Owner\n\nPurchaser
3,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Owner\n\nPurchaser
4,"Irwin Monk, et als\n\nHACA",Owner\n\nPurchaser


In [54]:
def splitRowEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    for i in range(len(table)):
        row = table.iloc[i]
        if '\n' in str(row['Decision People Involved']):
            data = row['Decision People Involved'].split('\n')
            roles= str(row['Decision Role']).split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            if('Heir' in roles):
                norm='Heir'
                norm_prev='Heir'
            for d in range(len(data)):
                newrow = row
                newrow['Decision People Involved'] = data[d]
                if data[d]!='':
                    newrow['Decision Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            size+=1
    return newframe

In [55]:
t=splitRowEvents(event_assoc)

In [56]:
t.head()

,Decision People Involved,Decision Role
0,NaN,NaN
1,Troy Barnett,Owner
2,Hattie Barnett,Owner
3,,Owner
4,HACA,Purchaser


In [57]:
t['Decision People Involved'].replace('', np.nan, inplace=True)

In [58]:
t.head()

,Decision People Involved,Decision Role
0,NaN,NaN
1,Troy Barnett,Owner
2,Hattie Barnett,Owner
3,NaN,Owner
4,HACA,Purchaser


In [59]:
t.dropna(inplace=True)

In [60]:
t.columns = ["people","role"]

In [61]:
# t.people=t.people.apply(lambda x: x.lower())

In [62]:
for index, row in t.iterrows():
    name = name_cleaner(str(row["people"]))
    t.loc[t.index == index,"people"] = name

In [63]:
t.head()
# NOTE: people 1-5 have incorrect roles 

,people,role
1,Troy Barnett,Owner
2,Hattie Barnett,Owner
4,HACA,Purchaser
5,Robert L Harris,Owner
6,Johnny J Wilson,Owner


In [64]:
t = t.merge(newframe,how='left', left_on='people',right_on='name')

In [65]:
t.dropna(inplace=True)

In [66]:
t.head()

,people,role,name,person_id
0,Troy Barnett,Owner,Troy Barnett,12925.0
1,Troy Barnett,Owner,Troy Barnett,12926.0
2,Troy Barnett,Owner,Troy Barnett,12927.0
3,Troy Barnett,Owner,Troy Barnett,12928.0
4,Hattie Barnett,Owner,Hattie Barnett,7784.0


In [67]:
newframe.to_csv('output/people_count.csv', index=False)

# Event Type

In [68]:
# extracting types of events 
types=['Previous Transfer of Deed','Final Transfer of Deed','Appraisal','Offer Made', 'Decision', 'Award','Tenant','End of Case']

In [69]:
event_types=pd.DataFrame({'Type':types})

In [70]:
event_types

,Type
0,Previous Transfer of Deed
1,Final Transfer of Deed
2,Appraisal
3,Offer Made
4,Decision
5,Award
6,Tenant
7,End of Case


In [71]:
event_types.insert(0, 'ID', range(1,1+len(event_types)))

In [72]:
event_types.to_csv('output/event_types.csv', index=False)

In [73]:
event_types.to_csv('db_output/event_types.csv', index=False)

# Events Table

1. Extract information about each event type (previous transfer, final transfer, award ...)
2. Clean each table to contain attributes needed for the database (date, money ...)   Exceptions are: (Block number and Parcel number) and (person, person ID) which are kept in the table for now, but will be changed to Parcel ID and Person ID later  
3. Concatenate all event types

## Event Type 1: Previous Transfer 

In [74]:
previous_events=copy[['Block #', 'Parcel #','Previous Transfer of Deed Event Date','Previous Transfer of Deed People Involved','Previous Transfer of Deed Role', 'Previous Transfer of Deed Notes' ]]

In [75]:
aa = str(previous_events.iloc[[44]]["Previous Transfer of Deed Event Date"])

In [76]:
def splitRowPreviousEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        
        parcel=row['Parcel #']
        block=row['Block #']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Previous Transfer of Deed People Involved']):
            
            # when the row is separated by ---        
            if '---' in str(row['Previous Transfer of Deed People Involved']):

                people = str(row['Previous Transfer of Deed People Involved'])
                roles = str(row['Previous Transfer of Deed Role'])
                agree_dates = str(row['Previous Transfer of deed Event Date'])
                
                people = re.sub(r'[\-]+', 'separator', people) 
                roles = re.sub(r'[\-]+', 'separator', roles) 
                agree_dates = re.sub(r'[\-]+', 'separator', agree_dates) 
                
                people = people.split("separator")
                roles = roles.split("separator")
                agree_dates = agree_dates.split("separator")
                
                prev_role = ""
                prev_date = ""
                
                for d in range(len(people)):
                    people2 = people[d].split('\n')
                    
                    # substitue characters by \n to then slit into new lines using \n
                    try:       
                        roles2 = re.sub(r'[\n]+', '\n', roles[d])                        
                        roles2 = roles2.strip().split('\n')
                        prev_role = roles2
                    except:
                        roles2 = prev_role
                        
                    try:                        
                        agree_dates2 = re.sub(r'[\n]+', '\n', agree_dates[d])                        
                        agree_dates2 = agree_dates2.strip().split('\n')
                        prev_date = agree_dates2
                    except:
                        agree_dates2 = prev_date                        
                    
                    
                    
                    role_counter = 0
                    date_counter = 0
                    
                    for k in range(len(people2)):
                        if people2[k] == '':
                            role_counter += 1
                            date_counter += 1

#                         name_cleaner: function which removes unwanted characters from the names 
                        newrow2 = row                
                        newrow2['Previous Transfer of Deed People Involved'] = name_cleaner(people2[k])
                            
                        try:
                            newrow2['Previous Transfer of Deed Role']= roles2[role_counter]
                        except:
                            role_counter -= 1
                            newrow2['Previous Transfer of Deed Role']= roles2[role_counter]

                        try:
                            newrow2['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])
                        except:
                            date_counter -= 1
                            newrow2['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])

                           
                        newrow2['Parcel #']=parcel
                        newrow2['Block #']=block                        

                        newframe.loc[size]=newrow2
                        size+=1
                
            # when the row is NOT separated by ---            
            else:
                roles = re.sub(r'[\n]+', '\n', str(row['Previous Transfer of Deed Role'])) 
                agree_dates = re.sub(r'[\n]+', '\n', str(row['Previous Transfer of Deed Event Date'])) 
                
                people = row['Previous Transfer of Deed People Involved'].split('\n')
                roles = roles.strip().split('\n')
                agree_dates = agree_dates.strip().split('\n')
                role_counter = 0
                date_counter = 0

                for d in range(len(people)):
                    if people[d] == '':
                        role_counter += 1
                        date_counter += 1

                    newrow = row                
                    newrow['Previous Transfer of Deed People Involved'] = name_cleaner(people[d])
                    try:
                        newrow['Previous Transfer of Deed Role']= roles[role_counter]
                    except:
                        role_counter -= 1
                        newrow['Previous Transfer of Deed Role']= roles[role_counter]
                        
                    try:
                        newrow['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                    except:
                        date_counter -= 1
                        newrow['Previous Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                        
                    newrow['Parcel #']=parcel
                    newrow['Block #']=block                        

                    newframe.loc[size]=newrow
                    size+=1
        else:
            if str(row["Previous Transfer of Deed People Involved"]) == "-" or str(row["Previous Transfer of Deed People Involved"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Previous Transfer of Deed People Involved"]))
                newframe.loc[newframe.index == size,"Previous Transfer of Deed People Involved"] = name
                date = date_cleaner(str(row["Previous Transfer of Deed Event Date"]))
                newframe.loc[newframe.index == size,"Previous Transfer of Deed Event Date"] = date
                
                
            size+=1
    return newframe


In [77]:
prev_events=splitRowPreviousEvents(previous_events)

In [78]:
prev_events['Previous Transfer of Deed People Involved'].replace('', np.nan, inplace=True)


In [79]:
prev_events=prev_events.dropna() 

In [80]:
prev_events

,Block #,Parcel #,Previous Transfer of Deed Event Date,Previous Transfer of Deed People Involved,Previous Transfer of Deed Role,Previous Transfer of Deed Notes
1,59,14,5/29/1946,A G Carver Jr,Owner,Listed as parcel 15.
2,59,14,5/29/1946,Evelyn Carter,Owner,Listed as parcel 15.
4,59,14,5/29/1946,Troy Barnett,Purchaser,Listed as parcel 15.
5,59,14,5/29/1946,Hattie Barnett,Purchaser,Listed as parcel 15.
7,59,15,10/13/1947,William P Shuferd,Owner,-
...,...,...,...,...,...,...
3949,33,7,6/9/1966,Marion B Haynes Jr,Owner,-
3951,33,7,6/9/1966,Wachovia Bank Trust Co,Trustee,-
3952,33,10,8/16/1943,Carolyn Thrash Dorsett,Owner,-
3954,33,10,8/16/1943,Lucile Renwick,Purchaser,-


In [81]:
prev_events.head()

,Block #,Parcel #,Previous Transfer of Deed Event Date,Previous Transfer of Deed People Involved,Previous Transfer of Deed Role,Previous Transfer of Deed Notes
1,59,14,5/29/1946,A G Carver Jr,Owner,Listed as parcel 15.
2,59,14,5/29/1946,Evelyn Carter,Owner,Listed as parcel 15.
4,59,14,5/29/1946,Troy Barnett,Purchaser,Listed as parcel 15.
5,59,14,5/29/1946,Hattie Barnett,Purchaser,Listed as parcel 15.
7,59,15,10/13/1947,William P Shuferd,Owner,-


In [82]:
prev_events.to_csv('output/prev_events.csv')

In [83]:
# Assigning Event ID to Previous Transfer of Deed (after cleaning up that data)

In [84]:
prev_events.insert(0, 'Event_Type', 1)

In [85]:
prev_events.columns = ["event_types", "block_num", "parcel_num", "event_date", "person", "person_role", "extra_information"]

In [86]:
prev_events.head()

,event_types,block_num,parcel_num,event_date,person,person_role,extra_information
1,1,59,14,5/29/1946,A G Carver Jr,Owner,Listed as parcel 15.
2,1,59,14,5/29/1946,Evelyn Carter,Owner,Listed as parcel 15.
4,1,59,14,5/29/1946,Troy Barnett,Purchaser,Listed as parcel 15.
5,1,59,14,5/29/1946,Hattie Barnett,Purchaser,Listed as parcel 15.
7,1,59,15,10/13/1947,William P Shuferd,Owner,-


In [87]:
prev_events.to_csv("output/prev_events.csv", index=False)

## Event Type 2: Final Transfer of Deed

In [88]:
final_transfer=copy[['Block #', 'Parcel #','Final Transfer of Deed Event Date','Final Transfer of Deed Amount Money','Final Transfer of Deed People Involved',
                   'Final Transfer of Deed Role', 'Final Transfer of Deed Notes']]

In [89]:
final_transfer.head()

,Block #,Parcel #,Final Transfer of Deed Event Date,Final Transfer of Deed Amount Money,Final Transfer of Deed People Involved,Final Transfer of Deed Role,Final Transfer of Deed Notes
0,34,7,NaN,NaN,NaN,NaN,NaN
1,59,14,12/4/1970,5000,Troy Barnett\nHattie Barnett\n\nHACA,Owner\n\nPurchaser,"Unclear title, including confusion with parcel..."
2,59,15,2/23/1971,6100,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,Owner\n\nPurchaser,"Deed lists Robert L. Harris, et als. Four name..."
3,59,16,7/26/1971,6500,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Owner\n\nPurchaser,Amount demanded after house was accidentally d...
4,59,18,6/3/1976,100,Irwin Monk\nW. Clarence Monk\nInez Monk\nFred ...,Owner\n\nPurchaser,Names taken from judgement of confirmation.


In [90]:
def splitRowFinalEvents(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Final Transfer of Deed Amount Money']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Final Transfer of Deed People Involved']):
            
            # when the row is separated by ---        
            if '---' in str(row['Final Transfer of Deed People Involved']):

                people = str(row['Final Transfer of Deed People Involved'])
                roles = str(row['Final Transfer of Deed Role'])
                agree_dates = str(row['Final Transfer of Deed Event Date'])
                
                people = re.sub(r'[\-]+', 'separator', people) 
                roles = re.sub(r'[\-]+', 'separator', roles) 
                agree_dates = re.sub(r'[\-]+', 'separator', agree_dates) 
                
                people = people.split("separator")
                roles = roles.split("separator")
                agree_dates = agree_dates.split("separator")
                
                prev_role = ""
                prev_date = ""
                
                for d in range(len(people)):
                    people2 = people[d].split('\n')
                    
                    try:                        
                        roles2 = re.sub(r'[\n]+', '\n', roles[d])                        
                        roles2 = roles2.strip().split('\n')
                        prev_role = roles2
                    except:
                        roles2 = prev_role
                        
                    try:                        
                        agree_dates2 = re.sub(r'[\n]+', '\n', agree_dates[d])                        
                        agree_dates2 = agree_dates2.strip().split('\n')
                        prev_date = agree_dates2
                    except:
                        agree_dates2 = prev_date                        
                    
                    
                    
                    role_counter = 0
                    date_counter = 0
                    
                    for k in range(len(people2)):
                        if people2[k] == '':
                            role_counter += 1
                            date_counter += 1

                        newrow2 = row                
                        newrow2['Final Transfer of Deed People Involved'] = name_cleaner(people2[k])
                            
                        try:
                            newrow2['Final Transfer of Deed Role']= roles2[role_counter]
                        except:
                            role_counter -= 1
                            newrow2['Final Transfer of Deed Role']= roles2[role_counter]

                        try:
                            newrow2['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])
                        except:
                            date_counter -= 1
                            newrow2['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates2[date_counter])

                           
                        newrow2['Parcel #']=parcel
                        newrow2['Block #']=block   
                        newrow2['Final Transfer of Deed Amount Money'] = money

                        newframe.loc[size]=newrow2
                        size+=1
                
            # when the row is NOT separated by ---            
            else:
                roles = re.sub(r'[\n]+', '\n', str(row['Final Transfer of Deed Role'])) 
                agree_dates = re.sub(r'[\n]+', '\n', str(row['Final Transfer of Deed Event Date'])) 
                
                people = row['Final Transfer of Deed People Involved'].split('\n')
                roles = roles.strip().split('\n')
                agree_dates = agree_dates.strip().split('\n')
                role_counter = 0
                date_counter = 0

                for d in range(len(people)):
                    if people[d] == '':
                        role_counter += 1
                        date_counter += 1

                    newrow = row                
                    newrow['Final Transfer of Deed People Involved'] = name_cleaner(people[d])
                    try:
                        newrow['Final Transfer of Deed Role']= roles[role_counter]
                    except:
                        role_counter -= 1
                        newrow['Final Transfer of Deed Role']= roles[role_counter]
                        
                    try:
                        newrow['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                    except:
                        date_counter -= 1
                        newrow['Final Transfer of Deed Event Date'] = date_cleaner(agree_dates[date_counter])
                        
                    newrow['Parcel #']=parcel
                    newrow['Block #']=block     
                    newrow['Final Transfer of Deed Amount Money'] = money

                    newframe.loc[size]=newrow
                    size+=1
        else:
            if str(row["Final Transfer of Deed People Involved"]) == "-" or str(row["Final Transfer of Deed People Involved"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Final Transfer of Deed People Involved"]))
                newframe.loc[newframe.index == size,"Final Transfer of Deed People Involved"] = name
                date = date_cleaner(str(row["Final Transfer of Deed Event Date"]))
                newframe.loc[newframe.index == size,"Final Transfer of Deed Event Date"] = date
                
                
            size+=1
    return newframe

In [91]:
final_transfer=splitRowFinalEvents(final_transfer)

In [92]:
final_transfer['Final Transfer of Deed People Involved'].replace('', np.nan, inplace=True)

In [93]:
final_transfer=final_transfer.dropna()

In [94]:
# Assigning Type_id 2
final_transfer.insert(0, 'Event_Type', 2)

In [95]:
final_transfer.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "person", "person_role", "extra_information"]

In [96]:
final_transfer.head()

,event_types,block_num,parcel_num,event_date,money,person,person_role,extra_information
1,2,59,14,12/4/1970,5000,Troy Barnett,Owner,"Unclear title, including confusion with parcel..."
2,2,59,14,12/4/1970,5000,Hattie Barnett,Owner,"Unclear title, including confusion with parcel..."
4,2,59,14,12/4/1970,5000,HACA,Purchaser,"Unclear title, including confusion with parcel..."
5,2,59,15,2/23/1971,6100,Robert L Harris,Owner,"Deed lists Robert L. Harris, et als. Four name..."
6,2,59,15,2/23/1971,6100,Johnny J Wilson,Owner,"Deed lists Robert L. Harris, et als. Four name..."


In [97]:
final_transfer.to_csv("output/final_transfer.csv", index=False)

## Event Type 3: Appraisal

In [98]:
appraisel=copy[['Block #', 'Parcel #','Appraiser Name','Appraiser Event Date','Appraiser Amount Money', 'Appraisal Notes']]

In [99]:
appraisel.head()

,Block #,Parcel #,Appraiser Name,Appraiser Event Date,Appraiser Amount Money,Appraisal Notes
0,34,7,NaN,NaN,NaN,NaN
1,59,14,Francis Naeger\nAlan Butterworth\nCharles W. B...,07/23/1968\n11/23/1966\n06/20/1966,4500\n5000\n3400,All appraisals list 22 Hibernia as parcel 15.
2,59,15,Francis Naeger\nAlan Butterworth\nCharles W. B...,09/11/1968\n11/17/1966\n09/15/1965,5600\n6100\n3050,Final title lists parcel 15; title report list...
3,59,16,Charles W. Bradshaw \nFrancis Naeger\nAlan But...,09/15/1965\n08/09/1966\n11/16/1966,2550\n5000\n4000,NaN
4,59,18,Roy Moore\nH.E. Johnson,05/16/1976\n05/13/1976,100\n100,-


In [100]:
exceptionList=[]
def splitRowAppraise(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        parcel=row['Parcel #']
        block=row['Block #']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Appraiser Name']):
            data = row['Appraiser Name'].split('\n')
            dates=str(row['Appraiser Event Date']).split('\n')
            money=str(row['Appraiser Amount Money']).split('\n')
            placeholder=0
            try:
                for d in range(len(data)):
                    newrow = row
                    newrow['Appraiser Name'] = name_cleaner(data[d])
                    newrow['Appraiser Event Date']= dates[d]
                    if d < len(money):
                        newrow['Appraiser Amount Money']=  money[d]
                        if placeholder!=0:
                            placeholder += 1
                    else:
                        newrow['Appraiser Amount Money']=  money[placeholder]
                    newrow['Parcel #']= parcel
                    newrow['Block #']= block
                    newframe.loc[size]=newrow
                    size+=1
            except:
                exceptionList.append(row) 
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Appraiser Name"]))
            newframe.loc[newframe.index == size,"Appraiser Name"] = name
            size+=1
    return newframe
 

In [101]:
appraisel = splitRowAppraise(appraisel)

In [102]:
appraisel.insert(0, 'Event_Type', 3)

In [103]:
print(exceptionList)

[Block #                                                                   5
Parcel #                                                                  2
Appraiser Name                                                             
Appraiser Event Date                                             09/15/1965
Appraiser Amount Money                                                 4400
Appraisal Notes           On the appraisal document, the name "Clyde G. ...
Name: 7, dtype: object, Block #                                                                  53
Parcel #                                                                15B
Appraiser Name                                                             
Appraiser Event Date                                             06/12/1974
Appraiser Amount Money                                                 1220
Appraisal Notes           On the appraisal document, the name "Clyde G. ...
Name: 11, dtype: object, Block #                               

In [104]:
appraisel.columns = ["event_types", "block_num", "parcel_num", "person", "event_date", "money", "ectra_information"]

In [105]:
appraisel.head()

,event_types,block_num,parcel_num,person,event_date,money,ectra_information
0,3,34,7,nan,NaN,NaN,NaN
1,3,59,14,Francis Naeger,07/23/1968,4500,All appraisals list 22 Hibernia as parcel 15.
2,3,59,14,Alan Butterworth,11/23/1966,5000,All appraisals list 22 Hibernia as parcel 15.
3,3,59,14,Charles W Bradshaw,06/20/1966,3400,All appraisals list 22 Hibernia as parcel 15.
4,3,59,15,Francis Naeger,09/11/1968,5600,Final title lists parcel 15; title report list...


In [106]:
appraisel.to_csv("output/appraisel_events.csv")

## Event Type 4: Offer Made

In [107]:
offerMade=copy[['Block #', 'Parcel #','Offer Made Amount Money','Offer Made People Involved','Offer Made Event Date','Offer Made Role', 'Offer Made Notes']]

In [108]:
offerMade.head()

,Block #,Parcel #,Offer Made Amount Money,Offer Made People Involved,Offer Made Event Date,Offer Made Role,Offer Made Notes
0,34,7,NaN,NaN,NaN,NaN,NaN
1,59,14,5000,Troy Barnett\nHattie Barnett\n\nHACA,3/17/1969,Owner\n\nPurchaser,NaN
2,59,15,6100,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,3/28/1969,Owner\n\nPurchaser,"01/31/1969 option signed by Charles F. Wilson,..."
3,59,16,5000,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,12/16/1969,Owner\n\nPurchaser,There is missing documents and therefore missi...
4,59,18,-,Irwin Monk\nW. Clarence Monk\nInez Monk\nFred ...,-,Owner\n\nPurchaser,-


In [109]:
def splitRowOfferMade(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date=row['Offer Made Event Date']
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Offer Made Amount Money']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Offer Made People Involved']):
            data = row['Offer Made People Involved'].split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            for d in range(len(data)):
                newrow = row
                newrow['Offer Made People Involved'] = name_cleaner(data[d])
                newrow['Offer Made Event Date']= date
                newrow['Parcel #']=parcel
                newrow['Block #']=block
                newrow['Offer Made Amount Money']=money
                if data[d]!='':
                    newrow['Offer Made Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Offer Made People Involved"]))
            newframe.loc[newframe.index == size,"Offer Made People Involved"] = name
            size+=1
    return newframe

In [110]:
offerMade=splitRowOfferMade(offerMade)
##there is an extra newline at the begining that messes up the first row

In [111]:
offerMade['Offer Made People Involved'].replace('', np.nan, inplace=True)

In [112]:
offerMade=offerMade.dropna()

In [113]:
offerMade.insert(0, 'Event_Type', 4)

In [114]:
offerMade.columns = ["event_types", "block_num", "parcel_num", "money", "person", "event_date", "person_role", "extra_information"]

In [115]:
offerMade.head()

,event_types,block_num,parcel_num,money,person,event_date,person_role,extra_information
5,4,59,15,6100,Robert L Harris,3/28/1969,Owner,"01/31/1969 option signed by Charles F. Wilson,..."
6,4,59,15,6100,Johnny J Wilson,3/28/1969,Owner,"01/31/1969 option signed by Charles F. Wilson,..."
7,4,59,15,6100,Leroy Harris,3/28/1969,Owner,"01/31/1969 option signed by Charles F. Wilson,..."
8,4,59,15,6100,Charles F Wilson,3/28/1969,Owner,"01/31/1969 option signed by Charles F. Wilson,..."
10,4,59,15,6100,HACA,3/28/1969,Purchaser,"01/31/1969 option signed by Charles F. Wilson,..."


In [116]:
offerMade.to_csv("output/offer_events.csv")

## Event Type 5: Decision Made

In [117]:
decision=copy[['Block #', 'Parcel #','Decision Event Date','Decision Amount Money','Decision Response','Decision People Involved','Decision Role', 'Decision Notes']]

In [118]:
decision.head()

,Block #,Parcel #,Decision Event Date,Decision Amount Money,Decision Response,Decision People Involved,Decision Role,Decision Notes
0,34,7,NaN,NaN,NaN,NaN,NaN,NaN
1,59,14,3/18/1969,5000,Accepted,Troy Barnett\nHattie Barnett\n\nHACA,Owner\n\nPurchaser,NaN
2,59,15,3/28/1969,6100,Accepted,Robert L. Harris\nJohnny J. Wilson\nLeroy Harr...,Owner\n\nPurchaser,-
3,59,16,12/16/1969,5000,Rejected,William I. Lee\nZylphin J. Lee\nH. Kenneth Lee...,Owner\n\nPurchaser,House was accidentally demolitioned in the int...
4,59,18,6/18/1976,100,Accepted,"Irwin Monk, et als\n\nHACA",Owner\n\nPurchaser,Condemnation.


In [119]:
def splitRowDecision(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date=row['Decision Event Date']
        parcel=row['Parcel #']
        block=row['Block #']
        money=row['Decision Amount Money']
        response=row['Decision Response']
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Decision People Involved']):
            data = row['Decision People Involved'].split('\n')
            norm='Owner'
            norm_prev='Owner'
            norm2='Purchaser'
            for d in range(len(data)):
                newrow = row
                newrow['Decision People Involved'] = name_cleaner(data[d])
                newrow['Decision Event Date']= date
                newrow['Parcel #']=parcel
                newrow['Block #']=block
                newrow['Decision Amount Money']=money
                newrow['Decision Response']=response
                if data[d]!='':
                    newrow['Decision Role']=norm_prev
                if data[d]=='':
                    if norm_prev==norm:
                        norm_prev=norm2
                    else:
                        norm_prev=norm
                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            newframe.loc[size]=row
            name = name_cleaner(str(row["Decision People Involved"]))
            newframe.loc[newframe.index == size,"Decision People Involved"] = name
            size+=1
    return newframe

In [120]:
decision=splitRowDecision(decision)

decision['Decision People Involved'].replace('', np.nan, inplace=True)

decision=decision.dropna()

decision.head()

,Block #,Parcel #,Decision Event Date,Decision Amount Money,Decision Response,Decision People Involved,Decision Role,Decision Notes
5,59,15,3/28/1969,6100,Accepted,Robert L Harris,Owner,-
6,59,15,3/28/1969,6100,Accepted,Johnny J Wilson,Owner,-
7,59,15,3/28/1969,6100,Accepted,Leroy Harris,Owner,-
8,59,15,3/28/1969,6100,Accepted,Charles F Wilson,Owner,-
10,59,15,3/28/1969,6100,Accepted,HACA,Purchaser,-


In [121]:
decision.insert(0,'Event_Type',5)

decision.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "response", "person", "person_role", "extra_information"]

decision.head()

,event_types,block_num,parcel_num,event_date,money,response,person,person_role,extra_information
5,5,59,15,3/28/1969,6100,Accepted,Robert L Harris,Owner,-
6,5,59,15,3/28/1969,6100,Accepted,Johnny J Wilson,Owner,-
7,5,59,15,3/28/1969,6100,Accepted,Leroy Harris,Owner,-
8,5,59,15,3/28/1969,6100,Accepted,Charles F Wilson,Owner,-
10,5,59,15,3/28/1969,6100,Accepted,HACA,Purchaser,-


In [122]:
decision.to_csv("output/decision_events.csv")

## Event Type 6: Award

In [123]:
award=copy[['Block #', 'Parcel #','Award Date','Award Amount Money', 'Award Notes']]

In [124]:
award.head()

,Block #,Parcel #,Award Date,Award Amount Money,Award Notes
0,34,7,NaN,NaN,NaN
1,59,14,-,-,NaN
2,59,15,-,-,-
3,59,16,-,-,-
4,59,18,-,-,-


In [125]:
award.insert(0,'Event_Type',6)

In [126]:
award.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "extra_information"]

In [127]:
award.head()

,event_types,block_num,parcel_num,event_date,money,extra_information
0,6,34,7,NaN,NaN,NaN
1,6,59,14,-,-,NaN
2,6,59,15,-,-,-
3,6,59,16,-,-,-
4,6,59,18,-,-,-


In [128]:
award.to_csv("output/award_events.csv")

## Event Type 7: Tenant

In [174]:
tenant=copy[['Block #', 'Parcel #','Tenant Vacated Date','Tenant Agreement Date','Tenant Cost','Tenant Name','Previous Owner or Tenant', 'Tenant Notes']]

In [175]:
tenant.head()

,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner or Tenant,Tenant Notes
0,34,7,NaN,NaN,NaN,NaN,NaN,NaN
1,59,14,-,-,-,-,-,-
2,59,15,-,-,-,-,-,-
3,59,16,-,-,-,-,-,-
4,59,18,-,-,-,-,-,-


### Needs to write a robust code to tease out names and their roles here

In [176]:
def splitRowTenant(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        date= row['Tenant Vacated Date'] if row['Tenant Vacated Date'] != "-" else ""
        agree_date= str(row['Tenant Agreement Date']) if str(row['Tenant Agreement Date']) != "-" else ""
        parcel=row['Parcel #']
        block=row['Block #']
#         money=row['Tenant Cost']
#         role=row['Previous owner or Tenant']
        
        ##Checks if there are any new line characters in the current row
        if '\n' in str(row['Tenant Name']):
            tenants = row['Tenant Name'].split('\n')
            rents = row['Tenant Cost'].split('\n')
            roles = row['Previous Owner or Tenant'].split('\n')
            agree_dates = str(row['Tenant Agreement Date']).split('\n')
            prev_role = ""
            prev_date = ""

            for d in range(len(tenants)):
                if tenants[d] == '':
                    continue
                
                newrow = row                
                newrow['Tenant Name'] = name_cleaner(tenants[d])
                
                try:
                    newrow['Previous Owner or Tenant']= roles[d]
                    prev_role = roles[d] 
                except:
                    newrow['Previous Owner or Tenant'] = prev_role
                
                try:
                    newrow['Tenant Agreement Date']= agree_dates[d]
                    prev_date = agree_date[d]
                except:
                    newrow['Tenant Agreement Date']= prev_date
                    
                newrow['Tenant Vacated Date']= date                
                newrow['Parcel #']=parcel
                newrow['Block #']=block                        
        
                newframe.loc[size]=newrow
                size+=1
        else:
            if str(row["Tenant Name"]) == "-" or str(row["Tenant Name"]) == "":
                newframe.loc[size]=row
            else:
                newframe.loc[size]=row
                name = name_cleaner(str(row["Tenant Name"]))
                newframe.loc[newframe.index == size,"Tenant Name"] = name
                
                try:
                    rent = re.sub(r'[\$\-]', '', str(row['Tenant Cost']))
                    newframe.loc[newframe.index == size,"Tenant Cost"] = rent
                except Exception as e: 
                    print(e)
                    print (row)
                    break
                
            size+=1
    return newframe

In [177]:
tenant.insert(0,'Event_Type',7)

In [178]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner or Tenant,Tenant Notes
0,7,34,7,NaN,NaN,NaN,NaN,NaN,NaN
1,7,59,14,-,-,-,-,-,-
2,7,59,15,-,-,-,-,-,-
3,7,59,16,-,-,-,-,-,-
4,7,59,18,-,-,-,-,-,-


In [179]:
tenant['Previous Owner or Tenant'].replace('', np.nan, inplace=True)
tenant=tenant.dropna()

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [180]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner or Tenant,Tenant Notes
1,7,59,14,-,-,-,-,-,-
2,7,59,15,-,-,-,-,-,-
3,7,59,16,-,-,-,-,-,-
4,7,59,18,-,-,-,-,-,-
5,7,59,19,-,-,-,-,-,-


In [181]:
tenant=splitRowTenant(tenant)

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying 

In [182]:
tenant['Tenant Name'].replace('', np.nan, inplace=True)
tenant=tenant.dropna()

In [183]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Agreement Date,Tenant Cost,Tenant Name,Previous Owner or Tenant,Tenant Notes
0,7,59,14,-,-,-,-,-,-
1,7,59,15,-,-,-,-,-,-
2,7,59,16,-,-,-,-,-,-
3,7,59,18,-,-,-,-,-,-
4,7,59,19,-,-,-,-,-,-


In [184]:
tenant.to_csv("output/split_tenant_events.csv", index= False)

In [185]:
tenant_notes= tenant[['Block #', 'Parcel #','Tenant Agreement Date', 'Tenant Notes']]

In [186]:
  def noteTitle(table):
    newframe=pd.DataFrame(columns=table.columns.values)
    size=0
    ##Goes through length of table
    for i in range(len(table)):
        ##Saves current row
        row = table.iloc[i]
        agreeDate= str(row['Tenant Agreement Date']) if str(row['Tenant Agreement Date']) != "-" else ""
        note= row['Tenant Notes'] if row['Tenant Notes'] != "-" else ""
        parcel=row['Parcel #']
        block=row['Block #']
        dateTitle= "Tenant Agreement Date: "
        noteTitle = "Other Notes: "
        
        
        if str(row["Tenant Agreement Date"]) == "-" or str(row["Tenant Agreement Date"]) == "":
                newframe.loc[size]=row
        else:
            newframe.loc[size]=row
            agreeDate = dateTitle + (str(row["Tenant Agreement Date"]))
            newframe.loc[newframe.index == size,"Tenant Agreement Date"] = agreeDate
                    
        if str(row["Tenant Notes"]) == "-" or str(row["Tenant Notes"]) == "":
                newframe.loc[size]=row
        else:
            newframe.loc[size]=row
            note = noteTitle + (str(row["Tenant Notes"]))
            newframe.loc[newframe.index == size,"Tenant Notes"] = note
                     
        size+=1
    return newframe

In [187]:
tenant_notes= noteTitle(tenant_notes)

In [188]:
tenant_notes.head()

,Block #,Parcel #,Tenant Agreement Date,Tenant Notes
0,59,14,-,-
1,59,15,-,-
2,59,16,-,-
3,59,18,-,-
4,59,19,-,-


In [189]:
tenant_notes["extra_information"] = tenant_notes["Tenant Agreement Date"] +" "+ tenant_notes["Tenant Notes"]

In [190]:
tenant_notes.head()

,Block #,Parcel #,Tenant Agreement Date,Tenant Notes,extra_information
0,59,14,-,-,- -
1,59,15,-,-,- -
2,59,16,-,-,- -
3,59,18,-,-,- -
4,59,19,-,-,- -


In [191]:
tenant_notes.to_csv("output/tenant_notes_test.csv", index= False)

In [192]:
tenant_notes.drop(['Tenant Agreement Date'], axis=1, inplace=True)

In [193]:
tenant_notes.drop(['Tenant Notes'], axis=1, inplace=True)

In [194]:
tenant.drop(['Tenant Notes'], axis=1, inplace=True)

In [195]:
tenant.drop(['Tenant Agreement Date'], axis=1, inplace=True)

In [196]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Cost,Tenant Name,Previous Owner or Tenant
0,7,59,14,-,-,-,-
1,7,59,15,-,-,-,-
2,7,59,16,-,-,-,-
3,7,59,18,-,-,-,-
4,7,59,19,-,-,-,-


In [197]:
tenant = tenant.merge(tenant_notes, how='left', left_on=['Block #','Parcel #'], right_on=['Block #','Parcel #'])

In [198]:
tenant.head()

,Event_Type,Block #,Parcel #,Tenant Vacated Date,Tenant Cost,Tenant Name,Previous Owner or Tenant,extra_information
0,7,59,14,-,-,-,-,- -
1,7,59,15,-,-,-,-,- -
2,7,59,16,-,-,-,-,- -
3,7,59,18,-,-,-,-,- -
4,7,59,19,-,-,-,-,- -


In [199]:
tenant.columns = ["event_types", "block_num", "parcel_num", "event_date", "money", "person", "person_role","extra_information"]


In [200]:
tenant.event_types = tenant["event_types"].astype(int)

In [201]:
tenant.head()

,event_types,block_num,parcel_num,event_date,money,person,person_role,extra_information
0,7,59,14,-,-,-,-,- -
1,7,59,15,-,-,-,-,- -
2,7,59,16,-,-,-,-,- -
3,7,59,18,-,-,-,-,- -
4,7,59,19,-,-,-,-,- -


In [202]:
tenant.to_csv("output/tenant_events.csv", index= False)

## Event Type 8: End of Case

In [139]:
endCase=copy[['Block #', 'Parcel #','End of Case Finalized Date']]

In [140]:
endCase.head()

,Block #,Parcel #,End of Case Finalized Date
0,34,7,NaN
1,59,14,12/4/1970
2,59,15,2/23/1971
3,59,16,7/26/1971
4,59,18,6/3/1976


In [141]:
endCase.insert(0,'Event_Type',8)

In [142]:
endCase.columns = ["event_types", "block_num", "parcel_num", "event_date"]

In [143]:
endCase.head()

,event_types,block_num,parcel_num,event_date
0,8,34,7,NaN
1,8,59,14,12/4/1970
2,8,59,15,2/23/1971
3,8,59,16,7/26/1971
4,8,59,18,6/3/1976


In [144]:
endCase.to_csv("output/end_events.csv")

## Concatenating all Events

In [145]:
prev_events.head()

,event_types,block_num,parcel_num,event_date,person,person_role
1,1,59,14,5/29/1946,A G Carver Jr,Owner
2,1,59,14,5/29/1946,Evelyn Carter,Owner
4,1,59,14,5/29/1946,Troy Barnett,Purchaser
5,1,59,14,5/29/1946,Hattie Barnett,Purchaser
7,1,59,15,10/13/1947,William P Shuferd,Owner


In [146]:
events_table = pd.concat([prev_events,final_transfer,appraisel,offerMade,decision,award,tenant, endCase,])

In [147]:
events_table.insert(0, 'event_id', range(1,1+len(events_table)))

In [148]:
events_table.head()

,event_id,event_types,block_num,parcel_num,event_date,person,person_role,money,response,extra_information
1,1,1,59,14,5/29/1946,A G Carver Jr,Owner,NaN,NaN,NaN
2,2,1,59,14,5/29/1946,Evelyn Carter,Owner,NaN,NaN,NaN
4,3,1,59,14,5/29/1946,Troy Barnett,Purchaser,NaN,NaN,NaN
5,4,1,59,14,5/29/1946,Hattie Barnett,Purchaser,NaN,NaN,NaN
7,5,1,59,15,10/13/1947,William P Shuferd,Owner,NaN,NaN,NaN


In [149]:
events_table.to_csv('output/events.csv')

# People Clean up (Bez)

Manually cleaned people_count.csv and created 3 tables, which are saved in Data forlder
1. table of script generated names and IDs (people_old.csv)
2. table of newly cleaned names and IDs (people_new.csv)
1. table associating the two tables via IDs (people_assoc.csv)


In [150]:
people_old = pd.read_csv("data/people_old.csv")

In [151]:
people_old.head()

,person_id,name
0,0,#NAME?
1,1,*Not Legable
2,2,A Blomberg
3,3,A C Brandl
4,4,A C Brandl Jr


In [152]:
people_new = pd.read_csv("data/people_new.csv")
people_new.head()

,Original name,new_person_id
0,#Name,0
1,*Not Legable,1
2,A Blomberg,2
3,A C Brandl,3
4,A C Brandl Jr,4


In [153]:
people_assoc = pd.read_csv("data/people_assoc.csv")
people_assoc.head()

,assoc_id,old_person_id,new_person_id
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4


In [ ]:
people_final = people_new.rename(columns={'new_person_id': 'person_id', 'Original name' : 'name'})

In [ ]:
#people_final.head()

In [ ]:
people_final.to_csv("db_output/people.csv", index=False)

# Join other tables based on the Event table

### Creating association table

a series of merges and drops, which will lead to the cleaned name matching with the correct event

In [186]:
##people_old.head()

In [187]:
##events_table.head()

In [203]:
events_association_table = events_table.merge(people_old, how='left', left_on='person', right_on='name')

In [185]:
##events_association_table.head()

In [204]:
events_association_table.drop(['name'], axis=1, inplace=True)

In [183]:
##events_association_table.head()

In [205]:
events_association_table = events_association_table.merge(people_assoc, how='left', left_on='person_id', right_on='old_person_id')

In [184]:
##events_association_table.head()

In [206]:
events_association_table.drop(['person_id'], axis=1, inplace=True)

In [207]:
events_association_table.drop(['assoc_id'], axis=1, inplace=True)

In [208]:
events_association_table = events_association_table.merge(people_new, how='left', left_on='new_person_id', right_on='new_person_id')

In [209]:
events_association_table.drop(['old_person_id'], axis=1, inplace=True)

In [210]:
events_association_table.drop(['person'], axis=1, inplace=True)

In [211]:
events_association_table = events_association_table.drop_duplicates()

In [212]:
events_association_table = events_association_table.rename(columns={'new_person_id': 'person_id', 'Original name' : 'name'})

In [213]:
events_association_table.to_csv("output/events_association_table.csv")

# event_people_ association table

In [214]:
event_people_assoc= events_association_table[["event_id", "person_id", "person_role"]]

In [215]:
event_people_assoc.insert(0, 'event_people_assoc_id', range(1,1+len(event_people_assoc)))

In [234]:
#event_people_assoc.head()

In [217]:
event_people_assoc.to_csv("output/event_people_assoc.csv")

In [218]:
event_people_assoc.to_csv("db_output/event_people_assoc.csv", index=False)

# events table

In [235]:
#events_association_table.head()

In [220]:
parcel = pd.read_csv("output/parcels.csv")

,parcel_id,block_num,parcel_num,ward_num,land_use
0,1,34,7,4,Commercial
1,2,59,14,4,Residential
2,3,59,15,4,Residential
3,4,59,16,4,Residential
4,5,59,18,4,Residential


In [ ]:
#parcel.head()

In [221]:
events_association_table = events_association_table.merge(parcel, how='left', left_on=['block_num','parcel_num'], right_on=['block_num','parcel_num'])

In [222]:
#events_association_table.head()

,event_id,event_types,block_num,parcel_num,event_date,person_role,money,response,extra_information,person_id,name,parcel_id,ward_num,land_use
0,1,1,59,14,5/29/1946,Owner,NaN,NaN,NaN,9.0,A G Carver Jr,2.0,4,Residential
1,2,1,59,14,5/29/1946,Owner,NaN,NaN,NaN,881.0,Evelyn G Roberts,2.0,4,Residential
2,3,1,59,14,5/29/1946,Purchaser,NaN,NaN,NaN,2876.0,Trinagle Investment Company,2.0,4,Residential
3,3,1,59,14,5/29/1946,Purchaser,NaN,NaN,NaN,2877.0,Troy A Barnett,2.0,4,Residential
4,4,1,59,14,5/29/1946,Purchaser,NaN,NaN,NaN,1203.0,Hattie Crowell Anderson Petty,2.0,4,Residential


In [223]:
events = events_association_table[["event_id", "response", "extra_information", "parcel_id","event_date","event_types","money", "person_id"]]

In [224]:
events["event_id"] = range(1,1+len(events))

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [225]:
#events.head()

,event_id,response,extra_information,parcel_id,event_date,event_types,money,person_id
0,1,NaN,NaN,2.0,5/29/1946,1,NaN,9.0
1,2,NaN,NaN,2.0,5/29/1946,1,NaN,881.0
2,3,NaN,NaN,2.0,5/29/1946,1,NaN,2876.0
3,4,NaN,NaN,2.0,5/29/1946,1,NaN,2877.0
4,5,NaN,NaN,2.0,5/29/1946,1,NaN,1203.0


In [197]:
events.to_csv("output/final_events.csv", index=False)

In [226]:
events.to_csv("db_output/events.csv", index=False)